In [1]:
#Livrarias utilizadas
import numpy as np
import theano
import theano.tensor as T
import lasagne
from helper import * #helper.py, funcoes para dataset e etc

Using gpu device 0: GeForce GT 630M (CNMeM is disabled)


In [2]:
def build_cnn(input_var=None):
    # Input layer
    network = lasagne.layers.InputLayer(shape=(None, 1, 28, 28),
                                        input_var=input_var)

    # Convolutional layer with 32 kernels of size 5x5. Strided and padded
    # convolutions are supported as well; see the docstring.
    network = lasagne.layers.Conv2DLayer(
            network, num_filters=32, filter_size=(5, 5),
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform())

    # Max-pooling layer of factor 2 in both dimensions:
    network = lasagne.layers.MaxPool2DLayer(network, pool_size=(2, 2))

    # Another convolution with 32 5x5 kernels, and another 2x2 pooling:
    network = lasagne.layers.Conv2DLayer(
            network, num_filters=32, filter_size=(5, 5),
            nonlinearity=lasagne.nonlinearities.rectify)
    network = lasagne.layers.MaxPool2DLayer(network, pool_size=(2, 2))

    # A fully-connected layer of 256 units with 50% dropout on its inputs:
    network = lasagne.layers.DenseLayer(
            lasagne.layers.dropout(network, p=.5),
            num_units=256,
            nonlinearity=lasagne.nonlinearities.rectify)

    # And, finally, the 10-unit output layer with 50% dropout on its inputs:
    network = lasagne.layers.DenseLayer(
            lasagne.layers.dropout(network, p=.5),
            num_units=10,
            nonlinearity=lasagne.nonlinearities.softmax)

    return network

In [4]:
num_epochs = 500

# Load the dataset
print("Loading data...")
X_train, y_train, X_val, y_val, X_test, y_test = load_mnist_dataset()

# Prepare Theano variables for inputs and targets
input_var = T.tensor4('inputs')
target_var = T.ivector('targets')

# Create neural network model (depending on first command line parameter)
print("Creating and compiling network")
network = build_cnn(input_var)

# Create a loss expression for training, i.e., a scalar objective we want
# to minimize (for our multi-class problem, it is the cross-entropy loss):
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean()

# We could add some weight decay as well here, see lasagne.regularization.

# Create update expressions for training, i.e., how to modify the
# parameters at each training step. Here, we'll use Stochastic Gradient
# Descent (SGD) with Nesterov momentum, but Lasagne offers plenty more.
params  = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(loss, params, learning_rate=0.01, momentum=0.9)

# Create a loss expression for validation/testing. The crucial difference
# here is that we do a deterministic forward pass through the network, 
# disabling dropout layers.
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(test_prediction, target_var)
test_loss = test_loss.mean()

# As a bonus, also create an expression for the classification accuracy:
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var), dtype=theano.config.floatX)

# Compile a function performing a training step on a mini-batch (by giving
# the updates dictionary) and returning the corresponding training loss:
train_fn = theano.function([input_var, target_var], loss, updates=updates)

# Compile a second function computing the validation loss and accuracy:
val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

# Finally, launch the training loop.
print("Starting training...")

# We iterate over epochs:
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    
    #iterate_minibatches defined in utils.py
    for batch in iterate_minibatches(X_train, y_train, 500, shuffle=True):
        inputs, targets = batch
        train_err += train_fn(inputs, targets)
        train_batches += 1

    # And a full pass over the validation data:
    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, 500, shuffle=False):
        inputs, targets = batch
        err, acc = val_fn(inputs, targets)
        val_err += err
        val_acc += acc
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    print("  validation accuracy:\t\t{:.2f} %".format(val_acc / val_batches * 100))

# After training, we compute and print the test error:    
test_err = 0
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500, shuffle=False):
    inputs, targets = batch
    err, acc = val_fn(inputs, targets)
    test_err += err
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))
print("  test accuracy:\t\t{:.2f} %".format(test_acc / test_batches * 100))

#Dump the network weights to a file
np.savez('model.npz', *lasagne.layers.get_all_param_values(network))
    

Loading data...
Creating and compiling network
Starting training...
Epoch 1 of 500 took 0.559s
  training loss:		2.337111
  validation loss:		2.311889
  validation accuracy:		10.00 %
Epoch 2 of 500 took 0.518s
  training loss:		2.348117
  validation loss:		2.307199
  validation accuracy:		10.00 %
Epoch 3 of 500 took 0.484s
  training loss:		2.316196
  validation loss:		2.302267
  validation accuracy:		10.60 %
Epoch 4 of 500 took 0.480s
  training loss:		2.318879
  validation loss:		2.297126
  validation accuracy:		11.60 %
Epoch 5 of 500 took 0.480s
  training loss:		2.304075
  validation loss:		2.292044
  validation accuracy:		12.00 %
Epoch 6 of 500 took 0.481s
  training loss:		2.301897
  validation loss:		2.287142
  validation accuracy:		12.60 %
Epoch 7 of 500 took 0.481s
  training loss:		2.292834
  validation loss:		2.282516
  validation accuracy:		14.80 %
Epoch 8 of 500 took 0.480s
  training loss:		2.285121
  validation loss:		2.278217
  validation accuracy:		15.80 %
Epoch 9 of 5

Epoch 72 of 500 took 0.479s
  training loss:		0.912343
  validation loss:		0.462158
  validation accuracy:		92.80 %
Epoch 73 of 500 took 0.480s
  training loss:		0.980781
  validation loss:		0.447183
  validation accuracy:		93.20 %
Epoch 74 of 500 took 0.479s
  training loss:		0.915926
  validation loss:		0.436814
  validation accuracy:		93.40 %
Epoch 75 of 500 took 0.479s
  training loss:		0.931296
  validation loss:		0.424846
  validation accuracy:		93.60 %
Epoch 76 of 500 took 0.480s
  training loss:		0.824037
  validation loss:		0.416334
  validation accuracy:		93.20 %
Epoch 77 of 500 took 0.480s
  training loss:		0.858199
  validation loss:		0.406684
  validation accuracy:		92.60 %
Epoch 78 of 500 took 0.481s
  training loss:		0.879284
  validation loss:		0.400091
  validation accuracy:		92.60 %
Epoch 79 of 500 took 0.481s
  training loss:		0.783005
  validation loss:		0.389319
  validation accuracy:		92.80 %
Epoch 80 of 500 took 0.478s
  training loss:		0.903020
  validation loss

Epoch 143 of 500 took 0.480s
  training loss:		0.508421
  validation loss:		0.207219
  validation accuracy:		96.20 %
Epoch 144 of 500 took 0.480s
  training loss:		0.471417
  validation loss:		0.207760
  validation accuracy:		96.00 %
Epoch 145 of 500 took 0.479s
  training loss:		0.472187
  validation loss:		0.208909
  validation accuracy:		96.00 %
Epoch 146 of 500 took 0.481s
  training loss:		0.533186
  validation loss:		0.210352
  validation accuracy:		96.20 %
Epoch 147 of 500 took 0.480s
  training loss:		0.469128
  validation loss:		0.210556
  validation accuracy:		96.60 %
Epoch 148 of 500 took 0.478s
  training loss:		0.509558
  validation loss:		0.214876
  validation accuracy:		96.40 %
Epoch 149 of 500 took 0.478s
  training loss:		0.491418
  validation loss:		0.212411
  validation accuracy:		96.40 %
Epoch 150 of 500 took 0.479s
  training loss:		0.504771
  validation loss:		0.213762
  validation accuracy:		96.40 %
Epoch 151 of 500 took 0.479s
  training loss:		0.428853
  valida

Epoch 214 of 500 took 0.479s
  training loss:		0.311212
  validation loss:		0.178548
  validation accuracy:		96.80 %
Epoch 215 of 500 took 0.479s
  training loss:		0.375859
  validation loss:		0.178141
  validation accuracy:		97.20 %
Epoch 216 of 500 took 0.479s
  training loss:		0.426024
  validation loss:		0.177580
  validation accuracy:		97.20 %
Epoch 217 of 500 took 0.479s
  training loss:		0.319732
  validation loss:		0.178681
  validation accuracy:		96.80 %
Epoch 218 of 500 took 0.480s
  training loss:		0.455625
  validation loss:		0.178173
  validation accuracy:		96.80 %
Epoch 219 of 500 took 0.479s
  training loss:		0.344664
  validation loss:		0.178334
  validation accuracy:		96.60 %
Epoch 220 of 500 took 0.479s
  training loss:		0.399033
  validation loss:		0.178295
  validation accuracy:		96.60 %
Epoch 221 of 500 took 0.479s
  training loss:		0.373748
  validation loss:		0.178463
  validation accuracy:		96.60 %
Epoch 222 of 500 took 0.479s
  training loss:		0.338902
  valida

Epoch 285 of 500 took 0.481s
  training loss:		0.308512
  validation loss:		0.169179
  validation accuracy:		97.40 %
Epoch 286 of 500 took 0.481s
  training loss:		0.340371
  validation loss:		0.168626
  validation accuracy:		97.20 %
Epoch 287 of 500 took 0.479s
  training loss:		0.298857
  validation loss:		0.168677
  validation accuracy:		97.20 %
Epoch 288 of 500 took 0.478s
  training loss:		0.321015
  validation loss:		0.169186
  validation accuracy:		97.20 %
Epoch 289 of 500 took 0.478s
  training loss:		0.285427
  validation loss:		0.169692
  validation accuracy:		97.00 %
Epoch 290 of 500 took 0.480s
  training loss:		0.322491
  validation loss:		0.170214
  validation accuracy:		97.20 %
Epoch 291 of 500 took 0.478s
  training loss:		0.326732
  validation loss:		0.169550
  validation accuracy:		97.40 %
Epoch 292 of 500 took 0.479s
  training loss:		0.318472
  validation loss:		0.169047
  validation accuracy:		97.20 %
Epoch 293 of 500 took 0.479s
  training loss:		0.257507
  valida

Epoch 356 of 500 took 0.520s
  training loss:		0.234902
  validation loss:		0.157816
  validation accuracy:		97.40 %
Epoch 357 of 500 took 0.520s
  training loss:		0.300528
  validation loss:		0.156555
  validation accuracy:		97.40 %
Epoch 358 of 500 took 0.520s
  training loss:		0.275033
  validation loss:		0.156408
  validation accuracy:		97.40 %
Epoch 359 of 500 took 0.520s
  training loss:		0.265888
  validation loss:		0.156215
  validation accuracy:		97.20 %
Epoch 360 of 500 took 0.520s
  training loss:		0.219190
  validation loss:		0.156186
  validation accuracy:		97.20 %
Epoch 361 of 500 took 0.520s
  training loss:		0.275223
  validation loss:		0.155804
  validation accuracy:		97.20 %
Epoch 362 of 500 took 0.520s
  training loss:		0.247973
  validation loss:		0.155905
  validation accuracy:		97.20 %
Epoch 363 of 500 took 0.520s
  training loss:		0.271289
  validation loss:		0.156202
  validation accuracy:		97.40 %
Epoch 364 of 500 took 0.520s
  training loss:		0.237544
  valida

Epoch 427 of 500 took 0.532s
  training loss:		0.228398
  validation loss:		0.152221
  validation accuracy:		97.40 %
Epoch 428 of 500 took 0.532s
  training loss:		0.281399
  validation loss:		0.151910
  validation accuracy:		97.40 %
Epoch 429 of 500 took 0.533s
  training loss:		0.249383
  validation loss:		0.152816
  validation accuracy:		97.20 %
Epoch 430 of 500 took 0.533s
  training loss:		0.213407
  validation loss:		0.152835
  validation accuracy:		97.20 %
Epoch 431 of 500 took 0.534s
  training loss:		0.185384
  validation loss:		0.153151
  validation accuracy:		97.40 %
Epoch 432 of 500 took 0.535s
  training loss:		0.259520
  validation loss:		0.152739
  validation accuracy:		97.20 %
Epoch 433 of 500 took 0.534s
  training loss:		0.271332
  validation loss:		0.153628
  validation accuracy:		97.40 %
Epoch 434 of 500 took 0.535s
  training loss:		0.243701
  validation loss:		0.153244
  validation accuracy:		97.40 %
Epoch 435 of 500 took 0.533s
  training loss:		0.258160
  valida

Epoch 498 of 500 took 0.545s
  training loss:		0.247673
  validation loss:		0.152101
  validation accuracy:		97.20 %
Epoch 499 of 500 took 0.544s
  training loss:		0.176412
  validation loss:		0.152407
  validation accuracy:		97.40 %
Epoch 500 of 500 took 0.543s
  training loss:		0.275359
  validation loss:		0.151441
  validation accuracy:		97.40 %
Final results:
  test loss:			0.125500
  test accuracy:		95.80 %
